**Created by Sanskar Hasija**

**🤖🤖H2O.ai AutoML Regression-House Price Prediction**

**15 NOVEMBER 2021**


# <center> 🤖🤖H2O.AI AUTOML REGRESSION-HOUSE PRICE PREDICTION</center>
## <center>If you find this notebook useful, support with an upvote👍</center>

# Imports

In [1]:
from IPython.display import clear_output

import numpy as np
import pandas as pd

import h2o
from h2o.automl import H2OAutoML

# Initializing H20 Cluster

In [2]:
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpom4zle_n
  JVM stdout: /tmp/tmpom4zle_n/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpom4zle_n/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,1 month and 8 days
H2O_cluster_name:,H2O_from_python_unknownUser_0ppfuc
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.250 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


# Data Loading and Preprocessing

In [3]:
train = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
test = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")
subs = pd.read_csv("../input/house-prices-advanced-regression-techniques/sample_submission.csv")

train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [4]:
train.describe()

Rows:1460
Cols:81




,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
type,int,int,enum,int,int,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,int,int,int,int,enum,enum,enum,enum,enum,int,enum,enum,enum,enum,enum,enum,enum,int,enum,int,int,int,enum,enum,enum,enum,int,int,int,int,int,int,int,int,int,int,enum,int,enum,int,enum,enum,int,enum,int,int,enum,enum,enum,int,int,int,int,int,int,enum,enum,enum,int,int,int,enum,enum,int
mins,1.0,20.0,,21.0,1300.0,,,,,,,,,,,,,1.0,1.0,1872.0,1950.0,,,,,,0.0,,,,,,,,0.0,,0.0,0.0,0.0,,,,,334.0,0.0,0.0,334.0,0.0,0.0,0.0,0.0,0.0,0.0,,2.0,,0.0,,,1900.0,,0.0,0.0,,,,0.0,0.0,0.0,0.0,0.0,0.0,,,,0.0,1.0,2006.0,,,34900.0
mean,730.5,56.89726027397261,,70.04995836802661,10516.828082191787,,,,,,,,,,,,,6.099315068493142,5.57534246575342,1971.2678082191774,1984.86575342466,,,,,,103.68526170798897,,,,,,,,443.639726027397,,46.549315068493144,567.2404109589045,1057.429452054793,,,,,1162.6267123287648,346.99246575342397,5.844520547945204,1515.463698630138,0.4253424657534246,0.057534246575342445,1.565068493150685,0.3828767123287676,2.866438356164384,1.0465753424657545,,6.517808219178076,,0.6130136986301383,,,1978.5061638868735,,1.7671232876712313,472.98013698630143,,,,94.24452054794516,46.66027397260276,21.954109589041092,3.4095890410958902,15.060958904109588,2.758904109589041,,,,43.489041095890414,6.321917808219178,2007.8157534246575,,,180921.1958904111
maxs,1460.0,190.0,,313.0,215245.0,,,,,,,,,,,,,10.0,9.0,2010.0,2010.0,,,,,,1600.0,,,,,,,,5644.0,,1474.0,2336.0,6110.0,,,,,4692.0,2065.0,572.0,5642.0,3.0,2.0,3.0,2.0,8.0,3.0,,14.0,,3.0,,,2010.0,,4.0,1418.0,,,,857.0,547.0,552.0,508.0,480.0,738.0,,,,15500.0,12.0,2010.0,,,755000.0
sigma,421.6100093688479,42.30057099381045,,24.28475177448321,9981.26493237915,,,,,,,,,,,,,1.3829965467415926,1.1127993367127316,30.202904042525297,20.64540680770938,,,,,,181.06620658721647,,,,,,,,456.0980908409278,,161.31927280654173,441.8669552924343,438.7053244594709,,,,,386.5877380410744,436.528435886257,48.62308143352018,525.4803834232024,0.5189106060898062,0.23875264627921167,0.5509158012954317,0.5028853810928912,0.8157780441442279,0.22033819838403076,,1.6253932905840511,,0.6446663863122297,,,24.689724768590267,,0.7473150101111095,213.8048414533803,,,,125.33879435172422,66.2560276766497,61.11914860172857,29.31733055678188,55.757415281874174,40.17730694453019,,,,496.12302445794415,2.7036262083595113,1.3280951205521143,,,79442.50288288663
zeros,0,0,,0,0,,,,,,,,,,,,,0,0,0,0,,,,,,861,,,,,,,,467,,1293,118,37,,,,,0,829,1434,0,856,1378,9,913,6,1,,0,,690,,,0,,81,81,,,,761,656,1252,1436,1344,1453,,,,1408,0,0,,,0
missing,0,0,0,259,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,81,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1.0,60.0,RL,65.0,8450.0,Pave,nan,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7.0,5.0,2003.0,2003.0,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856.0,854.0,0.0,1710.0,1.0,0.0,2.0,1.0,3.0,1.0,Gd,8.0,Typ,0.0,nan,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0.0,61.0,0.0,0.0,0.0,0.0,nan,nan,nan,0.0,2.0,2008.0,WD,Normal,208500.0
1,2.0,20.0,RL,80.0,9600.0,Pave,nan,Reg,L

# AutoML

In [5]:
RANDOM_STATE = 12
RUNTIME = 1800 #30 mins
N_FOLDS = 5

x = train.columns
y = "SalePrice"
x.remove(y)
x.remove('Id')

In [6]:
aml = H2OAutoML(nfolds=N_FOLDS, seed=RANDOM_STATE, max_runtime_secs=RUNTIME)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_4_AutoML_1_20211116_50123

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 238481573.42398208
RMSE: 15442.848617531095
MAE: 8089.385696741185
RMSLE: 0.07092001016840364
R^2: 0.9621865280250592
Mean Residual Deviance: 238481573.42398208
Null degrees of freedom: 1459
Residual degrees of freedom: 1455
Null deviance: 9207911334609.979
Residual deviance: 348183097199.01385
AIC: 32318.412320303105

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 712249722.2564694
RMSE: 26688.007086638547
MAE: 15136.911607008673
RMSLE: 0.12610445129800615
R^2: 0.8870661807324525
Mean Residual Deviance: 712249722.2564694
Null degrees of freedom: 1459
Residual degrees of freedom: 1455
Null deviance: 9214679010535

# Results Leaderboard

### Top 10 models

In [7]:
lb = aml.leaderboard
lb.head(10)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_4_AutoML_1_20211116_50123,7.1225e+08,26688,7.1225e+08,15136.9,0.126104
GBM_grid_1_AutoML_1_20211116_50123_model_81,7.21706e+08,26864.6,7.21706e+08,15613.2,0.129927
GBM_grid_1_AutoML_1_20211116_50123_model_40,7.26945e+08,26961.9,7.26945e+08,15712.8,0.130604
GBM_grid_1_AutoML_1_20211116_50123_model_17,7.27529e+08,26972.8,7.27529e+08,16030.5,0.133301
GBM_grid_1_AutoML_1_20211116_50123_model_4,7.35686e+08,27123.5,7.35686e+08,15668.6,0.130634
GBM_grid_1_AutoML_1_20211116_50123_model_1,7.36054e+08,27130.3,7.36054e+08,16024.6,0.132459
GBM_grid_1_AutoML_1_20211116_50123_model_42,7.38336e+08,27172.3,7.38336e+08,15786,0.131371
GBM_grid_1_AutoML_1_20211116_50123_model_26,7.50815e+08,27401,7.50815e+08,15994.9,0.132515
StackedEnsemble_BestOfFamily_3_AutoML_1_20211116_50123,7.52265e+08,27427.5,7.52265e+08,15548.2,0.129054
GBM_grid_1_AutoML_1_20211116_50123_model_76,7.55429e+08,27485.1,7.55429e+08,16231.4,0.13484


### Submission

In [8]:
preds = aml.leader.predict(test)
clear_output()
sub = preds[0].as_data_frame()  ## Converting to dataframe
subs['SalePrice'] = sub
subs.to_csv('h20.ai-automl.csv',index = False)
subs.head()

,Id,SalePrice
0,1461,120001.418574
1,1462,153638.047319
2,1463,183015.942978
3,1464,192486.489368
4,1465,191120.353649
